In [3]:
import logging.config
import pickle
from pathlib import Path
from pprint import pprint
import sys
import numpy as np
import pandas as pd
from dateutil.parser import parse
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from tqdm.autonotebook import tqdm
sys.path.append('..')
from IPython.display import display
import random
import math
from tqdm.autonotebook import tqdm
from dotenv import load_dotenv
from src.nuforc import SETTINGS
from src.nuforc.utility import *
from src.nuforc.geocoding.geocoder import Geolocator
from src.nuforc.geocoding.wrangling import replace_unparsed_with_none, join_columns
from datetime import date
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
from geopandas import GeoSeries
from scipy.stats import mode
load_dotenv()
import os
import hashlib

In [39]:
# Events
raw = pd.read_csv(Path(os.getenv('DATA_DIR')) / 'raw_events' / 'events_2023_08_20.csv')
display(raw)

,occurred_time,reported_time,entered_as_time,shape,duration,city,state,state_abbreviation,country,description,raw_text,hash
0,7/1/2021 00:08,6/30/2021 10:09:17,07/01/2021 0:08,light,0:01:00,Sterling,Virginia,VA,USA,"Two ruby red dots, appeared out of thin air an...",NUFORC Home Report Indexes : by Event Date ...,62be080f0b4d72ced5c4d76ad2cd23618373181164bbbc...
1,3/3/2022 19:30,2/11/2023 6:53:04,03/03/2022 7:30,circle,0:00:25,Nashville,Tennessee,TN,USA,Lighted up as passenger jet was flying past it...,NUFORC Home Report Indexes : by Event Date ...,3fa51d668419adfd8a3a1ed4ec738f47375edd272257f3...
2,3/1/2022 00:00,4/3/2022 8:18:30,03/01/2022 12:00,cylinder,0:00:20,Fort Payne,Alabama,AL,USA,"I'm capturing these ""rods"" on my infrared secu...",NUFORC Home Report Indexes : by Event Date ...,fd80e1c56185ef3dc70f66c4b6525348408071a5cfbfc7...
3,3/1/2022 01:30,3/3/2022 5:24:23,03/01/2022 1:30,disk,2:00:00,Daleville,Virginia,VA,USA,"Object moved in gridlike pattern for 2 hours, ...",NUFORC Home Report Indexes : by Event Date ...,0f776f839b9bbea155f54bb5507da2b7f89775aef58e64...
4,3/1/2022 01:27,3/2/2022 5:43:58,03/01/2022 1:27,triangle,0:00:12,Kearneysville,West Virginia,WV,USA,Only the edges of the craft were visible the c...,NUFORC Home Report Indexes : by Event Date ...,8c0272d74ed43b357701db2047db1ef2025669c020ad91...
5,3/1/2022 03:00,3/6/2022 5:32:31,03/01/2022 3:00,other,"2 days, 0:00:00",Wentzville,Missouri,MO,USA,"I seen space craft ,the bottom ,look like squa...",NUFORC Home Report Indexes : by Event Date ...,b842aa41f308f658621cd8e3a467b53494141771d4e3bd...
6,3/1/2022 03:00,3/8/2022 4:09:23,03/01/2022 3:00,,0:00:35,Wentzville,Missouri,MO,USA,"I seen space craft ,the bottom ,look like squa...",NUFORC Home Report Indexes : by Event Date ...,c7dc12c70bc13a9e58b720b44d008f52ab36c00adc80bb...
7,3/1/2022 17:00,5/8/2022 2:12:05,03/01/2022 5:00,,0:30:00,Huntsville,Alabama,AL,USA,"They were silent, my friend made a video of th...",NUFORC Home Report Indexes : by Event Date ...,ec8fda8169646f67a30e08763df9779699aabce35687dd...
8,3/1/2022 04:29,3/19/2022 9:03:28,03/01/2022 4:29,unparsed,unparsed,Star Valley,Arizona,AZ,USA,MADAR Node 53,NUFORC Home Report Indexes : by Event Date ...,807a3603356c303d68da10fb9a51ebbfea9c5356c2521f...
9,1/31/2022 21:40,1/31/2022 7:13:38,01/31/2022 9:40,triangle,0:02:00,Columbus,Indiana,IN,USA,"Triangular object hovering in one spot, dark i...",NUFORC Home Report Indexes : by Event Date ...,ab8d18156ede4b7450b0ee99cdb75ecce1af3de0e4227e...


In [34]:
df = df[['hash', 'city', 'country', 'description', 'duration',
       'entered_as_time', 'occurred_time', 'raw_text', 'reported_time',
       'shape', 'state', 'state_abbreviation']]

df

,hash,city,country,description,duration,entered_as_time,occurred_time,raw_text,reported_time,shape,state,state_abbreviation
0,467d2d169ec61d52e1f14194db78f309438773916e38bd...,O'Fallon,USA,I walked outside with my wife to smoke a cigar...,0:00:05,10/01/2021 3:35,10/1/2021 03:35,NUFORC Home Report Indexes : by Event Date ...,10/2/2021 1:45:47,Disk,Missouri,MO
1,4e8017ddb77573964ce38fdf038c0fc535fd0f5b224377...,IMPERIAL BEACH,USA,"6-5-4-3-2-1, 1-2-3-4-5-6, RIGHT TRIANGLE, ACUT...",1:30:00,06/27/2022 9:16,6/27/2022 21:16,NUFORC Home Report Indexes : by Event Date ...,6/28/2022 5:06:08,Light,Rhode Island,RI
2,053cac51843e1875234981eca7278a4beff291b4a15da4...,San Diego,USA,Viewed two bright lights changing color over O...,0:02:00,06/27/2022 9:08,6/27/2022 21:08,NUFORC Home Report Indexes : by Event Date ...,6/27/2022 9:13:39,Sphere,California,CA
3,8c2289b9cc3845734805694714ba891ab826030954d182...,Livermore,USA,The dots moved into a particular line and The ...,0:00:05,06/27/2022 9:24,6/27/2022 21:24,NUFORC Home Report Indexes : by Event Date ...,6/27/2022 10:24:06,Changing,California,CA
4,ce879c33491d0a906c0ac880883a6ab3a6dea60610742c...,Walla Walla,USA,While shooting time-lapse Astrophotography I s...,0:01:00,06/24/2022 9:58,6/24/2022 21:58,NUFORC Home Report Indexes : by Event Date ...,6/25/2022 1:18:46,Circle,Washington,WA
...,...,...,...,...,...,...,...,...,...,...,...,...
145209,35ba6d83e312ac5f0aab92cd6326bbf94ff33e3fc185b3...,Winston salem,USA,4 orbs dissipated into 2 then disappeared See...,0:04:30,07/08/2023 9:43,7/8/2023 21:43,NUFORC Home Report Indexes : by Event Date ...,7/9/2023 7:36:40,Fireball,North Carolina,NC
145210,faa394b917fe5c54f85b9f0fbb68f3715687ed85834d51...,Moundridge,USA,Rectangular object traveled east north east ov...,0:00:03,07/09/2023 10:40,7/9/2023 22:40,NUFORC Home Report Indexes : by Event Date ...,7/9/2023 9:09:40,Other,Kansas,KS
145211,14354d3487c19d8a1eb67e81a3f9d1bbced228bcfaaa81...,North Myrtle Beach,USA,Light appeared about 25 ft above my head and m...,0:00:45,07/09/2023 11:15,7/9/2023 23:15,NUFORC Home Report Indexes : by Event Date ...,7/10/2023 6:53:07,Light,Ohio,OH
145212,e44fe0f3469d113b88326c1ea47b3d225d57f7690385a0...,Melbourne,USA,I open back door and saw a light in sky hoveri...,0:00:15,07/09/2023 11:54,7/9/2023 23:54,NUFORC Home Report Indexes : by Event Date ...,7/9/2023 9:29:10,Circle,Florida,FL


In [5]:
def hash_string(s):
    hash_object = hashlib.sha256()
    hash_object.update(s.encode())
    return hash_object.hexdigest()


In [18]:
# Apply the hashing function to the 'text' column with a progress bar
tqdm.pandas()
df['hash'] = df['raw_text'].progress_apply(lambda x: hash_string(x))

  0%|          | 0/145214 [00:00<?, ?it/s]

In [19]:
df.iloc[0]['hash']

'467d2d169ec61d52e1f14194db78f309438773916e38bda82b1c2752de9cb364'

In [35]:
df.to_csv(Path(os.getenv('DATA_DIR')) / 'raw_events' / 'events_2023_07_21.csv', index=False)